In [10]:
import numpy as np
import cv2
from PIL import Image
import copy
import matplotlib.pyplot as plt
import os
from utils import helper
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [3]:
# Create dataset
# Columns: image_path, label_path, bboxes, classes
imlabel_list = helper.imlabel('data/kitti/custom_annotated/annotated_images/', 'data/kitti/custom_annotated/labels/')

In [4]:
imlabel_list[0][0]

'data/kitti/custom_annotated/annotated_images/2-wheeler-1.jpg'

In [5]:
df = pd.DataFrame(columns = ['image_path','label_path','class','bboxes'])
# df = pd.DataFrame()
# df['image_path'] = imlabel_list[:][0]
# df['label_path'] = imlabel_list[:][1]
# df.head()

for item in imlabel_list:
    bboxes = helper.fetch_bboxes(item[1]).tolist()
    classes = helper.fetch_classes(item[1])
    for i,cls_bbox in enumerate(zip(classes,bboxes)):
        # print(cls_bbox)
        #df.loc[i,'image_path'] = item[0]
        #df.loc[i,'label_path'] = item[1]
        #df.loc[i,'class'] = cls_bbox[0]
        
        #df.loc[i,'bboxes'] = cls_bbox[1]
        # df = df.append([item[0],item[1],cls_bbox[0],cls_bbox[1]])
        df = df.append({'image_path':item[0], 'label_path': item[1],
                        'class': cls_bbox[0], 'bboxes': cls_bbox[1]}, ignore_index=True)
df.head()        

,image_path,label_path,class,bboxes
0,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[181.0, 83.0, 343.0, 396.0, 6.93056707688527e-..."
1,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[557.0, 47.0, 691.0, 377.0, 6.93063381931377e-..."
2,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[322.0, 116.0, 439.0, 348.0, 6.93056707689555e..."
3,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[426.0, 122.0, 514.0, 295.0, 6.930633801793e-310]"
4,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,2-wheeler,"[125.0, 134.0, 167.0, 221.0, 1.03e-321]"


In [7]:
df['class'].unique()

class_dict = {'2-wheeler': 0, 
              'pedestrian': 1, 'car': 2,
              'dontcare': 3}

df['class'] = df['class'].apply(lambda x:  class_dict[x])
df.head()

,image_path,label_path,class,bboxes
0,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,0,"[181.0, 83.0, 343.0, 396.0, 6.93056707688527e-..."
1,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,0,"[557.0, 47.0, 691.0, 377.0, 6.93063381931377e-..."
2,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,0,"[322.0, 116.0, 439.0, 348.0, 6.93056707689555e..."
3,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,0,"[426.0, 122.0, 514.0, 295.0, 6.930633801793e-310]"
4,data/kitti/custom_annotated/annotated_images/2...,data/kitti/custom_annotated/labels/2-wheeler-1...,0,"[125.0, 134.0, 167.0, 221.0, 1.03e-321]"


In [11]:
def normalize(im_arr):
    # Normalizes image with imagenet stats."""
    im_arr = im_arr/255
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im_arr - imagenet_stats[0])/imagenet_stats[1]

class Kitti(Dataset):
    def __init__(self, paths, bboxes, y):
        # self.transforms = transforms
        self.paths = paths.values
        self.bboxes = bboxes.values
        self.y = y.values
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        y_class = self.y[idx]
        y_bbox = self.bboxes[idx]
        # x, y_bb = transformsXY(path, self.bb[idx], self.transforms)
        x = normalize(x)
        x = np.rollaxis(x, 2)
        return x, y_class, y_bbox